In [14]:
import cv2
from PIL import Image
import keras
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array,array_to_img
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential,Model
from keras.layers import Dense,Conv2D,Flatten,Dropout
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
import random

In [2]:
categories = ['Krishnendu_Nag','Pralayesh_Nag','Sourav_Nag','Archana_Nag','Dipali_Nag']
num_class = len(categories)
height,width,depth = 100,100,3
datadir = 'C:/Users/SOURAV NAG/Pictures/Camera Roll/'
data=[]
i=0
for c in categories:
    path = os.path.join(datadir,c)
    label = categories.index(c)
    for img in os.listdir(path):
        try:
            image=os.path.join(path,img)
            img_arr=cv2.imread(image)
            img_arr=cv2.resize(img_arr,(width,height))
            data.append([img_arr,label])
            i+=1
            #print(i)
        except Exception as e:
            pass

In [3]:
random.shuffle(data)
X=[]
y=[]
for img,label in data:
    X.append(img)
    y.append(label)
X=np.array(X)
y=np.array(y)
print(y[:10])

[0 0 3 2 2 2 2 0 3 1]


In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

X_train=X_train.reshape(X_train.shape[0],height,width,depth)
X_test=X_test.reshape(X_test.shape[0],height,width,depth)
    
input_shape = (height,width,depth)

In [5]:
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
X_train/=255
X_test/=255

from keras.utils import to_categorical
y_train=to_categorical(y_train,num_class)
y_test=to_categorical(y_test,num_class)

In [6]:
vgg = VGG16(input_shape=input_shape,weights='imagenet',include_top=False)

for layer in vgg.layers:
    layer.trainable = False
    
next_layer = Flatten()(vgg.output)

prediction = Dense(num_class,activation='softmax')(next_layer)

model = Model(inputs=vgg.input, outputs = prediction)
model.summary()

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])







_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
____

In [7]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=16,epochs=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 388 samples, validate on 98 samples
Epoch 1/2
388/388 [==============================] - 69s 177ms/step - loss: 0.6643 - acc: 0.8325 - val_loss: 0.1010 - val_acc: 1.0000
Epoch 2/2
388/388 [==============================] - 69s 177ms/step - loss: 0.0678 - acc: 1.0000 - val_loss: 0.0355 - val_acc: 1.0000


In [8]:
 FaceClassifier = cv2.CascadeClassifier('C:/Users/SOURAV NAG/Desktop/Face_Recognition/haarcascade_frontalface_default.xml')

In [9]:
def face_extractor(img):
    faces = FaceClassifier.detectMultiScale(img,1.3,5)
    
    if faces is ():
        print('....')
        return None
    
    for (x,y,w,h) in faces:
        #print(x,y,w,h)
        cv2.rectangle(img,(x,y),(x+w,y+h),(200,10,50),2)
        cropped_face = img[y:y+h,x:x+w]
    
    return cropped_face

In [12]:
vc = cv2.VideoCapture(0)

while(vc.isOpened()):
    ret,frame = vc.read()
    if ret==True:
        #frame = np.array(frame, dtype=np.uint8)
        face = face_extractor(frame)
        
        if type(face) is np.ndarray:
            face  = cv2.resize(face,(width,height))
            im = Image.fromarray(face, 'RGB')
            im_array = np.array(im)
            #im_array = np.expand_dims(im_array,axis=0)
            im_array = np.reshape(im_array,(1,height,width,depth))
            im_array=im_array.astype('float32')
            im_array = im_array/255
            pred = model.predict(im_array)
            #print(pred)
            pred = np.argmax(pred[0],axis=0)
            print(pred)
            name = 'None Matching'
            
            if(pred == 0):
                name = 'Krishnendu'
            elif(pred == 1):
                name = 'Pralayesh'
            elif(pred == 2):
                name = 'Sourav'
            elif(pred == 3):
                name = 'Archana Nag'
            elif(pred == 4):
                name = 'Dipali Nag'
            elif(pred == 5):
                name = 'Badal Nag'
                
            cv2.putText(frame,name,(30,30),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2)
        else:
            cv2.putText(frame,'No Face Found',(30,30),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2)
        
        cv2.imshow('video',frame)
        if cv2.waitKey(1)==13:
            break
    else:
        break
        
vc.release()
cv2.destroyAllWindows()

2
2
4
4
4
4
4
2
2
4
4
4
4
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
....
....
....
....
....
....
....
....
....
....
....
....
....
....
0
0
0
0
0
0
0
0
0
0
4
4
4
4
2
2
2
4
4
0
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
4
4
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
0
2
2
2
2
4
4
4
4
2
0
0
2
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
2
4

2
2
4
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
2
2
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....
....


In [13]:
#!pip install h5py
from keras.models import model_from_json

model_json = model.to_json()
with open("Face_model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("Face_model1.h5")

json_file = open('Face_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("Face_model1.h5")
 
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])